In [40]:
import pandas as pd
import time
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [41]:
from numba import njit
from window_ops.expanding import expanding_mean
from window_ops.rolling import rolling_mean

@njit
def rolling_mean_14(x):
    return rolling_mean(x, window_size=14)
@njit
def rolling_mean_30(x):
    return rolling_mean(x, window_size=30)

In [42]:
def format_df_to_mlforecast(df, date_col, target_col, unique_id='mean'):
    df_ = df.rename({
        date_col: "ds",
        # target_col: 'y',
    }, axis=1)

    df_['ds'] = pd.to_datetime(df_['ds'])

    df_['y'] = df_[target_col].copy()
    # df_.drop(columns=target_col)

    df_['unique_id'] = unique_id
    return df_

In [43]:
selected_sensors_df = pd.read_csv("../data/selected_sensors2_cleaned.csv", index_col=0)

In [44]:
TEST_START_DATE = "2019-04-02"
scenarios_sensors = {
    # 0: 1, 4372603
    # "0_12M_train_7M_test": {"train_start": "2017-03-25", "train_end": "2018-03-25", "test_start": "2018-03-26", "test_end": "2018-10-10"},
    '2': {
        # "18M_train":  {"train_start": "2017-04-01", "train_end": "2018-10-01"},
        "12M_train":  {"train_start": "2017-04-01", "train_end": "2018-04-01", "val_start": "2017-04-01", "val_end": "2018-04-01"},
        # "12M_train_3M_val":  {"train_start": "2017-04-01", "train_end": "2018-04-01", "val_start": "2018-04-01", "val_end": "2018-07-01"},
        "12M_train_6M_val":  {"train_start": "2017-04-01", "train_end": "2018-04-01", "val_start": "2018-04-01", "val_end": "2018-10-01"},
        "12M_train_9M_val":  {"train_start": "2017-04-01", "train_end": "2018-04-01", "val_start": "2018-04-01", "val_end": "2019-01-01"},
        "12M_train_12M_val":  {"train_start": "2017-04-01", "train_end": "2018-04-01", "val_start": "2018-04-01", "val_end": "2019-04-01"},
        },
}
scenarios_sensors['5'] = scenarios_sensors['2'].copy()
scenarios_sensors['6'] = scenarios_sensors['2'].copy()

In [45]:
from MLForecastPipeline import *

In [46]:
def full_split_data(df, scenario, test_start_date=TEST_START_DATE, date_col="ds"):
    """Extracts train and test data based on train end date."""
    train_data = df[df[date_col] <= scenario['train_end']]
    val_data = df[(df[date_col] > scenario['val_start']) & (df[date_col] <= scenario['val_end'])]
    test_data = df[df[date_col] >= test_start_date]
    return train_data, val_data, test_data

models = {
    "SGD_Ridge": SGDRegressor( penalty='l2', alpha=1, random_state=42 ),
    "SGDRegressor": SGDRegressor(random_state=42),
    "SGD_ElasticNet": SGDRegressor( penalty='elasticnet', l1_ratio=0.5, alpha=0.001, random_state=42 ),
}

# Define lag transformations
from mlforecast.lag_transforms import *
lag_transforms_options = [
    # {},
    {1: [rolling_mean_14], 7: [rolling_mean_30], 30: [expanding_mean]},
    {1: [expanding_mean], 7: [rolling_mean_14], 30: [expanding_mean]},
    # {7: [RollingMean(window_size=7)], 30: [RollingMean(window_size=30)], 60: [RollingMean(window_size=60)], },
    {7: [RollingMean(7), RollingStd(7)], 30: [RollingMean(30)], 60: [ExpandingMean()], 14: [ExponentiallyWeightedMean(alpha=0.3)],},
    {7: [RollingMean(7), RollingStd(7), ExpandingStd()], 14: [RollingMean(14), ExpandingStd(), ExponentiallyWeightedMean(alpha=0.3)], 30: [RollingMean(30)], 60: [ExpandingMean()],},
]

In [47]:
# Reshaping to MLForecast format
def format_multi_df_to_mlforecast(df):
    df_melted = df.melt(id_vars=['full_date'], var_name='unique_id', value_name='y')
    return df_melted.rename(columns={'full_date': 'ds'})

In [48]:
def optuna_objective(trial, train_df, test_df, transforms, lags, lag_transforms):
    alpha = trial.suggest_float('alpha', 1e-6, 1, log=True)
    l1_ratio = trial.suggest_float('l1_ratio', 0.0, 1.0)
    max_iter = trial.suggest_int('max_iter', 300, 1000, step=100)  # Optimizing max_iter (number of iterations)
    eta0 = trial.suggest_float('eta0', 1e-6, 1, log=True)
    tol = trial.suggest_loguniform('tol', 1e-6, 1e-3)

    model = SGDRegressor(alpha=alpha, l1_ratio=l1_ratio, max_iter=max_iter, eta0=eta0, tol=tol, penalty='elasticnet', random_state=42)

    try:
        fcst = MLForecast(
            models=[model],
            freq='D',
            lags=lags,
            target_transforms=transforms,
            lag_transforms=lag_transforms,
            num_threads=1,
        )
        fcst.fit(train_df)
        predictions = fcst.predict(h=len(test_df))
        mape = mape_met(test_df['y'].values, predictions['SGDRegressor'].values)
        return mape
    except Exception as e:
        print(e)
        return float('inf')
    
import optuna

def run_optuna_search(train_df, test_df, transforms, lags, lag_transforms, n_trials=30):
    study = optuna.create_study(direction='minimize')
    study.optimize(lambda trial: optuna_objective(trial, train_df, test_df, transforms, lags, lag_transforms), n_trials=n_trials)
    return study.best_params


In [ ]:
sensor_name = '2'
scenario = scenarios_sensors['2']['12M_train']
ratios = [1]

formatted_df = format_df_to_mlforecast(selected_sensors_df[['full_date', sensor_name]], 'full_date', sensor_name, unique_id=sensor_name)
formatted_df = formatted_df[['ds', 'y', 'unique_id']]

train_df, val_df, test_df = full_split_data(formatted_df, scenario)
optimal_lags_list = get_optimal_lags(train_df, 'y', ratios=ratios)
target_transforms = get_dynamic_transforms(train_df)

In [59]:
len(optimal_lags_list) * len(valid_transform_combinations) * 42 * len(lag_transforms_options)

8064

In [50]:
valid_transform_combinations = [()] + list(chain(combinations(target_transforms, 1), combinations(target_transforms, 2)))
valid_transform_combinations = [tc for tc in valid_transform_combinations if filter_conflicting_transforms(tc)]

transforms = list(valid_transform_combinations[1])
lags = optimal_lags_list[list(optimal_lags_list.keys())[0]]
lag_transforms = lag_transforms_options[0]

In [ ]:
best_params = run_optuna_search(train_df, val_df, transforms, lags, lag_transforms, n_trials=42)

optuna_model = SGDRegressor(**best_params, random_state=42)
models['SGD_Optuna'] = optuna_model

[I 2025-04-08 22:47:02,124] A new study created in memory with name: no-name-542460a6-16a5-49a2-beb2-eee722bcb573


C:\Users\77019\AppData\Local\Temp\ipykernel_21844\2493118097.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  tol = trial.suggest_loguniform('tol', 1e-6, 1e-3)
[I 2025-04-08 22:47:12,842] Trial 0 finished with value: 62.88467248209314 and parameters: {'alpha': 9.01339696590819e-05, 'l1_ratio': 0.17783879305086248, 'max_iter': 1000, 'eta0': 4.371466975720943e-05, 'tol': 0.0006440191774605649}. Best is trial 0 with value: 62.88467248209314.
C:\Users\77019\AppData\Local\Temp\ipykernel_21844\2493118097.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  tol = trial.suggest_loguniform('tol', 1e-6, 1e-3)
[I 2025-04-08 22:47:16,142] Trial 1 finished with value: 60.774611459

Input X contains NaN.
SGDRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values


c:\Users\77019\pyver\py312\Lib\site-packages\sklearn\utils\extmath.py:208: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
[I 2025-04-08 22:47:28,322] Trial 6 finished with value: inf and parameters: {'alpha': 0.0012892100651971882, 'l1_ratio': 0.6270811485009592, 'max_iter': 700, 'eta0': 0.4243675499043197, 'tol': 2.5782293544137194e-06}. Best is trial 1 with value: 60.77461145928027.
C:\Users\77019\AppData\Local\Temp\ipykernel_21844\2493118097.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  tol = trial.suggest_loguniform('tol', 1e-6, 1e-3)
c:\Users\77019\pyver\py312\Lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


Input X contains infinity or a value too large for dtype('float64').


...

Input X contains infinity or a value too large for dtype('float64').


c:\Users\77019\pyver\py312\Lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
[I 2025-04-08 22:48:51,663] Trial 38 finished with value: 70.77793640944564 and parameters: {'alpha': 1.0403272521575172e-05, 'l1_ratio': 0.24723945161976274, 'max_iter': 400, 'eta0': 3.471362458157527e-06, 'tol': 0.0006356411278043431}. Best is trial 33 with value: 58.622184898355115.
C:\Users\77019\AppData\Local\Temp\ipykernel_21844\2493118097.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  tol = trial.suggest_loguniform('tol', 1e-6, 1e-3)
[I 2025-04-08 22:48:54,990] Trial 39 finished with value: 64.55308832663594 and parameters: {'alpha': 1.0109835771209514e-06, 'l1_ratio': 0.37788327287

Input X contains infinity or a value too large for dtype('float64').


c:\Users\77019\pyver\py312\Lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1616: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
[I 2025-04-08 22:49:11,222] Trial 45 finished with value: 59.613145471038735 and parameters: {'alpha': 0.0006741408444255351, 'l1_ratio': 0.5862279953012113, 'max_iter': 900, 'eta0': 4.673156679636657e-06, 'tol': 0.0001579777654408446}. Best is trial 40 with value: 58.258383093733116.
C:\Users\77019\AppData\Local\Temp\ipykernel_21844\2493118097.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  tol = trial.suggest_loguniform('tol', 1e-6, 1e-3)
[I 2025-04-08 22:49:14,312] Trial 46 finished with value: 68.45742235967958 and parameters: {'alpha': 2.1511141310128827e-06, 'l1_ratio': 0.650264109207

Input X contains NaN.
SGDRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values


In [ ]:
from joblib import Parallel, delayed
import time

def process_scenario(sensor_name, scenario_name, scenario, selected_sensors_df, models, lag_transforms_options, ratios=[0.33, 0.66, 1]):
    """ Process each scenario independently and save results. """
    print(f'{sensor_name}_{scenario_name}')
    formatted_df = format_df_to_mlforecast(selected_sensors_df[['full_date', sensor_name]], 'full_date', sensor_name, unique_id=sensor_name)
    formatted_df = formatted_df[['ds', 'y', 'unique_id']]
    
    train_df, test_df = split_data(formatted_df, scenario)
    optimal_lags_list = get_optimal_lags(train_df, 'y', ratios=ratios)
    target_transforms = get_dynamic_transforms(train_df)

    results = evaluate_models(train_df, test_df, models, target_transforms, lag_transforms_options, optimal_lags_list)

    # Save results
    save_results(results, f"results/run_18/{sensor_name}_{scenario_name}.csv")

    return results

def run_all_scenarios_parallel(scenarios_sensors, selected_sensors_df, models, lag_transforms_options, ratios=[0.33, 0.66, 1]):
    # don't use all cpus (instead all but one)
    results = Parallel(n_jobs=15)( 
        delayed(process_scenario)(sensor_name, scenario_name, scenario, selected_sensors_df, models, lag_transforms_options, ratios=ratios)
        for sensor_name, scenarios in scenarios_sensors.items()
        for scenario_name, scenario in scenarios.items()
    )

    return results
